Wine Quality Data Set:
https://archive.ics.uci.edu/ml/datasets/Wine+Quality

# Predicting the Quality of Red Wines

By: Jaskaran Singh, Justin Wong, Matthew Gillies, Shehab Khan

# Introduction

For those who drink wine, knowing the quality of the wine is often very important to them. In our study, we look at four factors to predict whether the quality of wine out of 10. Our data set contains 1599 red wines with data on their acidity, composition, density, and quality. First, we will identify the 4 factors with the highest correlations to quality. To ensure that these factors do have a postive correlation with the quality of the wine, we will perform a hypothesis test and obtain a confidence interval regarding the correlatons. Using these factors we will predict the quality of the wines.

-2 parameters?
-frame question based on papers.

# Preliminary Results

In [2]:
library(cowplot)
library(digest)
library(gridExtra)
library(repr)
library(tidyverse)
library(infer)
library(tidymodels)
library(GGally)


Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.1.1     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::combine() masks gridExtra::combine()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()



ERROR: Error in library(tidymodels): there is no package called ‘tidymodels’


In [ ]:
wine_quality <- read_delim('winequality-red.csv', show_col_types = FALSE)
names(wine_quality) <- make.names(names(wine_quality), unique = TRUE)


head(wine_quality)

### UNIVARIATE ANALYSIS

For this analysis, we took bootstrap samples for each of the variables individually to check which of them deem neccessary for our analysis. 

In [ ]:
# We check first for wine_quality 
wine_quality_bootstrap <- wine_quality %>% 
    rep_sample_n(size = 40, reps = 2000, replace = TRUE)

wine_quality_hist_fixed <- wine_quality_bootstrap %>% 
    ggplot(aes(x = fixed.acidity)) + 
    #geom_histogram(aes(y = ..density.. ), colour = "#FF6666", fill = "white", binwidth = 0.8) + 
    geom_density(alpha=.2, fill="#e34444", colour = "#FF6666") 

wine_quality_hist_volatile <- wine_quality_bootstrap %>% 
    ggplot(aes(x = volatile.acidity)) + 
    #geom_histogram(aes(y = ..density.. ), colour = "#FF6666", fill = "white", binwidth = 0.1) + 
    geom_density(alpha=.2, fill="#e34444", colour = "#FF6666")

wine_quality_hist_citric <- wine_quality_bootstrap %>% 
    ggplot(aes(x = citric.acid)) + 
    #geom_histogram(aes(y = ..density.. ), colour = "#FF6666", fill = "white", binwidth = 0.04) + 
    geom_density(alpha=.2, fill="#e34444", colour = "#FF6666")

wine_quality_hist_sulfur <- wine_quality_bootstrap %>% 
    ggplot(aes(x = free.sulfur.dioxide)) + 
    #geom_histogram(aes(y = ..density.. ), colour = "#FF6666", fill = "white", binwidth = 0.8) + 
    geom_density(alpha=.2, fill="#e34444", colour = "#FF6666")

wine_quality_hist_sulfur_total <- wine_quality_bootstrap %>% 
    ggplot(aes(x = total.sulfur.dioxide)) + 
    #geom_histogram(aes(y = ..density.. ), colour = "#FF6666", fill = "white", binwidth = 0.8) + 
    geom_density(alpha=.2, fill="#e34444", colour = "#FF6666")

wine_quality_hist_chlorides <- wine_quality_bootstrap %>% 
    ggplot(aes(x = chlorides)) + 
    #geom_histogram(aes(y = ..density.. ), colour = "#FF6666", fill = "white", binwidth = 0.8) + 
    geom_density(alpha=.2, fill="#e34444", colour = "#FF6666")

wine_quality_hist_density <- wine_quality_bootstrap %>% 
    ggplot(aes(x = density)) + 
    #geom_histogram(aes(y = ..density.. ), colour = "#FF6666", fill = "white", binwidth = 0.02) + 
    geom_density(alpha=.2, fill="#e34444", colour = "#FF6666")

wine_quality_hist_ph <- wine_quality_bootstrap %>% 
    ggplot(aes(x = pH)) + 
    #geom_histogram(aes(y = ..density.. ), colour = "#FF6666", fill = "white", binwidth = 0.04) + 
    geom_density(alpha=.2, fill="#e34444", colour = "#FF6666")

wine_quality_hist_sulphates <- wine_quality_bootstrap %>% 
    ggplot(aes(x = sulphates)) + 
    #geom_histogram(aes(y = ..density.. ), colour = "#FF6666", fill = "white", binwidth = 0.8) + 
    geom_density(alpha=.2, fill="#e34444", colour = "#FF6666")

wine_quality_hist_alcohol <- wine_quality_bootstrap %>% 
    ggplot(aes(x = alcohol)) + 
    #geom_histogram(aes(y = ..density.. ), colour = "#FF6666", fill = "white", binwidth = 0.07) + 
    geom_density(alpha=.2, fill="#e34444", colour = "#FF6666")

head(wine_quality_bootstrap)

all_plots_row <- plot_grid(wine_quality_hist_fixed + 
                           theme(axis.text.x = element_text(angle = 90)),

                           wine_quality_hist_volatile +  
                           theme(axis.text.x = element_text(angle = 90)), 
                        
                           wine_quality_hist_citric +
                           theme(axis.text.x = element_text(angle = 90)), 

                           wine_quality_hist_sulfur + 
                           theme(axis.text.x = element_text(angle = 90)),
                           
                           wine_quality_hist_sulfur_total + 
                           theme(axis.text.x = element_text(angle = 90)),
                           
                           wine_quality_hist_chlorides + 
                           theme(axis.text.x = element_text(angle = 90)),
                           
                           wine_quality_hist_density + 
                           theme(axis.text.x = element_text(angle = 90)),
                           
                           wine_quality_hist_ph + 
                           theme(axis.text.x = element_text(angle = 90)),
                           
                           wine_quality_hist_alcohol + 
                           theme(axis.text.x = element_text(angle = 90)),
                           ncol = 2)

means_grid <- plot_grid(all_plots_row, 
                        rel_heights = c(0.5, 1))
    
means_grid

Based on the univariate graphs, we can see that on taking bootstrap samples, fixed acidity and volatile acidity follow a somewhat normal distribution wherein the histogram for volatile acidity follows a somewhat bimodal distribution. 

It can be seen that pH and density follows an almost perfect normal distribution while total and free sulfur dioxide follow a right skewed distribution. This suggests that the variables fixed acidity, volatile acidity, pH, and density should be our ideal variables for our analysis to predict the quality of the wine. 



In [ ]:
# Now we split data into a 75:25 training and testing data, which is then scaled and centered 
# to make sure all variables are comparable

wine_split <- initial_split(wine_quality, prop = 0.75, strata = quality)
wine_train <- training(wine_split)
wine_test <- testing(wine_split)

wine_recipe <- recipe(quality ~ . , data = wine_train) %>%
                step_scale(all_predictors()) %>%
                step_center(all_predictors())

wine_scaled <- wine_recipe %>%
            prep() %>%
            bake(wine_train)


In [3]:
options(repr.plot.width = 15, repr.plot.height = 12)

ggpairs(wine_scaled) + 
     ggtitle("Figure 1: Correlation Matrix Between Pairs of Variables")

ERROR: Error in ggpairs(wine_scaled): could not find function "ggpairs"


# Methods and Results

# References


 Zhang, Qing-An, et al. “Preliminary Study of the Effect of Ultrasound on Physicochemical Properties of Red Wine.” CyTA - Journal of Food, vol. 14, no. 1, 2015, pp. 55–64., https://doi.org/10.1080/19476337.2015.1045036. 

P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. Modeling wine preferences by data mining from physicochemical properties.
In Decision Support Systems, Elsevier, 47(4):547-553, 2009.

P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.
Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.